In [ ]:
!pip install -qU crewai crewai_tools langchain-google-genai python-dotenv serpapi

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = ""
from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import SerpApiGoogleSearchTool
os.environ["GOOGLE_API_KEY"] = ""
my_llm = LLM(model="gemini/gemini-2.5-flash", temperature=0.2)
print("✅ CrewAI Installed.")

In [ ]:
search_tool = SerpApiGoogleSearchTool()

# 1. Agents
researcher = Agent(
    role="Researcher",
    goal="Fetch latest market news and current stock price",
    tools=[search_tool],
    llm=my_llm,
    verbose=True,
    backstory = "You are an expert in Stock Market Research"
)

analyst = Agent(
    role="Analyst",
    goal="Extract the top 3 key insights from raw research data",
    llm=my_llm,
    backstory="You are a strategic analyst who filters noise and finds patterns.",
    verbose=True
)

writer = Agent(
    role="Executive Writer",
    goal="Write a concise, professional executive briefing",
    llm=my_llm,
    backstory="You write clear, decision-ready summaries for senior leadership.",
    verbose=True
)

# 1. Tasks
research_task = Task(
    description=(
        "Search the web for recent articles, reports, and discussions about {topic}. "
        "Return only raw facts, headlines, and data."
    ),
    expected_output="Bullet points of raw findings with sources.",
    agent=researcher
)

analysis_task = Task(
    description=(
        "Analyze the research data and identify the top 3 key trends or insights."
    ),
    expected_output="Three concise insights with brief explanations.",
    agent=analyst,
    context = [research_task]
)

writing_task = Task(
    description=(
        "Convert the insights into a one-page executive summary in markdown format."
    ),
    expected_output="A polished markdown briefing ready for leadership review.",
    agent=writer,
    context = [analysis_task]
)


In [ ]:
crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task],
    process=Process.sequential
)

# 🟢 DEMO TIME: Run this cell to show the linear handoff.
# result = pipeline_crew.kickoff()
# print("Pipeline Result:", result)

result = crew.kickoff(
    inputs={"topic": "AI Trends in 2025"}
)

print(result)